# Python言語によるビジネスアナリティクス
## 実務家のための最適化，統計分析，機械学習（近代科学社）

## statsmodelsを用いた統計の例題と練習問題


### 線形回帰

http://logopt.com/data/Diamond.csv からダイアモンドの価格データを読み込み，線形回帰を適用する．

列は ["carat","colour","clarity","certification","price"] であり，他の情報から価格(price)の予測を行う．

1. データをpandasのデータフレームとして読み込む．
2. statsmodels.formula.apiを **smf** (stats model formula)の名前でインポートする．
3. smfの一般化線形モデル**glm**を用いてモデルインスタンスを生成する．
このとき，列名を用いた**式(formula)**を文字列で記述し引数**formula**で，データは引数**data**にデータフレームとして入力する．
4. モデルインスタンスの**fitメソッド**で学習を行う．
5. モデルインスタンスの**summary(2)**メソッドで結果を見る．

モデル式の**formula**の書き方については，教科書194ページを参照

In [ ]:
import pandas as pd
%matplotlib inline
diamond = pd.read_csv('http://logopt.com/data/Diamond.csv', index_col=0)
diamond.head()

In [ ]:
import statsmodels.formula.api as smf
model = smf.glm('price ~ carat + colour + clarity +certification', diamond) 
fit = model.fit()                                
print(fit.summary2())    

### サマリーの見方

- No. Observations : サンプル数 (=308)
- Df Model : 自由度(Degree of Freedom) 変数の数なので $12$   
- AIC : [赤池情報量基準(Akaike Information Criterion) ](https://ja.wikipedia.org/wiki/%E8%B5%A4%E6%B1%A0%E6%83%85%E5%A0%B1%E9%87%8F%E8%A6%8F%E6%BA%96)
($=4931.3248 = -2 \times 対数尤度 + 2 \times (自由度+1) = -2 \times (-2452.7) +2 \times (12+1)$ )　(小さいほどモデルの適合度が良い）
- Log-Likelihood: 尤度の対数（最大尤度のものを求めている）[最尤推定](https://ja.wikipedia.org/wiki/%E6%9C%80%E5%B0%A4%E6%8E%A8%E5%AE%9A)
- Corf. : 係数（一番上のInterceptはy切片）
- Std. Err. : 標準誤差
- z : 標準偏差．大きいほど係数が信頼できる
- P : P値（偶然|z|を超える確率）．小さいほど係数が信頼できる（以下の表参照）
- [0.025, 0.975] : 係数の信頼区間

| z (標準偏差) | P値 (確率） | 信頼度 |
|:-----------|------------:|:------------:|
| < -1.65 または > +1.65 | < 0.10 | 90%
| < -1.96 または > +1.96 | < 0.05 | 95%
| < -2.58 または > +2.58 |  < 0.01 | 99%  


In [ ]:
diamond['predict'] = fit.predict() #予測を行い，結果を'predict'列に追加
diamond.plot.scatter(x='predict',y='price'); #描画

### 問題

http://logopt.com/data/carprice.csv から車の価格データを読み込み，線形回帰による予測を行え．

車種(Type)，100マイル走る際のガロン数（gpm100），都市部での1ガロンあたりの走行距離（MPGcity），高速道路での１ガロン当たりの走行距離（MPGhighway）から，価格(Price)を予測せよ．

### 問題

広告のデータ http://logopt.com/data/Advertising.csv を読み込み，線形回帰による予測を行え．

テレビ(TV)，ラジオ(Radio)，新聞(Newspaper)への広告から売り上げ(Sales)を予測せよ．

### 問題

http://logopt.com/data/Boston.csv のBostonの住宅データを用いて回帰分析を行え．

データの詳細については，
https://archive.ics.uci.edu/ml/datasets/Housing
を参照せよ．

medvが住宅の価格で，他のデータ（犯罪率や人口など）から予測する．

### 問題

http://logopt.com/data/SATGPA.csv データを用いて，2種類のSATの成績からGPAを予測せよ．

### 一般化線形モデルについて

基本となる線形回帰だと，独立変数 $x^{(i)}$ を用いて従属変数 $y^{(i)}$ を推定する．上付き添え字の$(i)$ はトレーニングデータのインデックスを表す．評価関数は最小自乗誤差であり，それを最小にするような重みベクトル $w$ を求める．

通常の線形回帰（最小自乗モデル）は，一般化線形モデル的に見直すと以下のように解釈できる．

1. 従属変数 $y^{(i)}$ は平均 $\mu^{(i)}$，標準偏差 $\sigma$ の正規分布 $N(\mu^{(i)},\sigma^2)$ にしたがう．
2. 線形予測子 $z^{(i)}$ を独立変数 $x^{(i)}$ を用いて $z^{(i)} = w x^{(i)} $ と定義する．ここで $w$ は最適化するパラメータ（重み）である．
3. リンク関数 $g$ を用いて  $\mu^{(i)}$ と  $z^{(i)}$ を繋ぐが，線形モデルでは $g(\mu) =\mu$ である．

### ロジスティック回帰

titanicデータを用いる(データの詳細については教科書82ページ参照）．

従属変数（予測するもの）は*survived*の列で，生き残ったか($=1$)，否か($=0$)を表す．

このように $0$ か $1$ かを予測するのに線形回帰は不適当なので，ロジスティック回帰を用いる．

一般化線形モデル(glm)を使えば，ほぼ同じように予測できる（性別sexと客室クラスpclassだけを用いる）．

引数の*family* に *sm.families.Binomial()* を指定すれば良い．

一般化線形モデルでの仮定は以下のようになる．

1. 従属変数 $y^{{i}}$ は平均 $\mu^{(i)}$ （表が出る確率）のコイン投げの分布(2項分布:binomial distribution)にしたがう．
2. 線形予測子 $z^{(i)}$ を独立変数 $x^{(i)}$ を用いて $z^{(i)} = w x^{(i)} $ と定義する．(この部分は全部共通）
3. リンク関数 $g$ を用いて  $\mu^{(i)}$ と  $z^{(i)}$ を繋ぐが，$\mu$ は確率なので $[0,1]$ の範囲しかとらない，一方， $z$ は線形予測子なので $[-\infty,+\infty]$ の定義域をもつ．これを繋ぐために以下のリンク関数 $g$ を用いる．

$$z = g(\mu) = \log \left( \frac{\mu}{1-\mu} \right) $$

これをロジット関数とよぶ．
歴史的な都合で $g$ は $\mu$ から $z$ への写像となっているが，逆写像として考えた方がわかりやすい．すなわち，線形予測子 $z$ から分布の平均 $\mu$ を逆写像 $g^{-1}$ で写すのである．この関数は


$$\mu = \frac{ \exp (z) }{ 1+\exp (z)} $$ となり，いわゆるロジスティック関数である．


### 例題

titanic号で生存したか否かの[データセット](https://www.kaggle.com/c/titanic/data)にロジスティック回帰を適用してみる．


In [ ]:
titanic = pd.read_csv('http://logopt.com/data/titanic.csv',index_col=0)
titanic.head()

In [ ]:
import statsmodels.api as sm
model = smf.glm(formula="Survived ~ Sex + Pclass + Fare + SibSp + Parch", data=titanic, family= sm.families.Binomial() )
res = model.fit() #学習
print(res.summary2())

In [ ]:
titanic['predict'] = res.predict()  #予測をpredict列に保管
titanic.plot.scatter(x='predict',y='Survived'); # 散布図に描画

### 問題

"http://logopt.com/data/cancer.csv" にある胸部癌か否かを判定するデータセットを用いて分類を行え．

最初の列diagnosisが癌か否かを表すものであり，'M'が悪性（malignant），'B'が良性（benign）である．

必要なら以下の文字列を切り貼りして用いよ．

formula = """diagnosis~radius_mean+texture_mean+texture_mean+perimeter_mean+area_mean+smoothness_mean+compactness_mean+
concavity_mean+symmetry_mean+radius_worst+texture_worst+perimeter_worst+area_worst+smoothness_worst+
compactness_worst+concavity_worst+symmetry_worst+fractal_dimension_worst"""

In [ ]:
cancer = pd.read_csv("http://logopt.com/data/cancer.csv", index_col=0)
cancer.head()

### 問題

"http://logopt.com/data/hospital.csv" にある病院のデータを用いてロジスティック回帰を行え．

従属変数*died*は死亡したか否かを表し，これを年齢(age)，施術(procedure)，性別(gender)，救急か否か(type)，入院日数(los: length of stay)から予測する． 

必要なら以下の文字列を使用しても良い．

formula="died~procedure+age+gender+los+type" 


In [ ]:
hospital = pd.read_csv("http://logopt.com/data/hospital.csv", index_col=0)
hospital.head()

### Poisson回帰

Poisson回帰は救急車の出動回数などの負の値をとらない**カウントデータ**もしくはその発生率を予測する際に用いられる．

この場合には，従属変数が $0$ 以上の値になるので，一般化線形モデルでの仮定は以下のようになる．

1. 従属変数 $y^{{i}}$ は平均 $\mu^{(i)}$ のPoisson分布にしたがう．
2. 線形予測子 $z^{(i)}$ を独立変数 $x^{(i)}$ を用いて $z^{(i)} = w x^{(i)} $ と定義する．(この部分は全部共通）
3. リンク関数 $g$ を用いて  $\mu^{(i)}$ と  $z^{(i)}$ を繋ぐが，$\mu$ は $0$ 以上で $z$ は $[-\infty,+\infty]$ の定義域をもつ．これを繋ぐために以下のリンク関数 $g$ を用いる．

$$z = g(\mu) = \log (\mu)$$

$g$ の逆写像は指数関数
$$ \mu = \exp (z) $$ である．

一般化線形モデル(glm)を使えば，ほぼ同じように予測できる．

引数の*family* に *sm.families.Poisson()* を指定すれば良い．

### 例題

"http://logopt.com/data/hospital-stay.csv" にある病院の入院日数のデータセットを用いてPoisson回帰を解説する．
従属変数である*los*（入院日数：length of stay)を，性別(gender)，救急か否か(type1)，75歳以上か(age75)から予測する．

入院日数は負の値をとらない，いわゆるカウントデータであるので，Poisson回帰を適用する．

In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
%matplotlib inline

In [ ]:
hospital = pd.read_csv("http://logopt.com/data/hospital-stay.csv",index_col=0)
hospital.head()

In [ ]:
model = smf.glm(formula="los ~ gender + type1 + age75 ", data=hospital, family= sm.families.Poisson() )
res = model.fit()
print(res.summary2())

### 問題
"http://logopt.com/data/fishing.csv" にある魚の数を予測するためのデータセットにPoisson回帰を適用せよ．
従属変数は魚の数を表す totabundであり，それを密度(density)，平均深度(meandepth)，年度(year)から予測せよ．

必要なら以下の文字列を用いよ．

formula="totabund ~ density + meandepth + year "


In [ ]:
fish = pd.read_csv("http://logopt.com/data/fishing.csv",index_col=0)
fish.head()